# Glue Spark

- RDD: Datasets which storage data in distributed format
- Transformations: Datasets iterations
- Actions: Run the transformations on RDD
- Output data: Result of transformations

Spark runs lazy, so transformations are not immediately executed, they stay in a logical plan to be executed when the actions are called.

Actions examples:

- `take()`
- `collect()`
- `show()`
- `save()`
- `count()`

## Types of transformations

### Narrow

Act only on one partition of the output data (Map, Filter)

- Does not requires to share data within another workers
- Are independent of other partitions
- Are computationally efficient

### Wide

Can work on several partitions of output data (Join, GroupBy)

## Running locally

1. Docker Compose file:

```yaml
services:
  glue:
    image: amazon/aws-glue-libs:glue_libs_4.0.0_image_01
    container_name: glue-jupyter
    ports:
      - "8889:8888"
      - "4040:4040"
    volumes:
      - .:/home/glue_user/workspace/jupyter_workspace
      - ~/.aws:/home/glue_user/.aws:ro
    working_dir: /home/glue_user/workspace
    environment:
      AWS_PROFILE: default
      DISABLE_SSL: true
    command: >
      /home/glue_user/jupyter/jupyter_start.sh
```

2. Run image:

```sh
docker compose run --rm glue
```

3. Connect to jupyter server within vscode:

- Select kernel
- Existing jupyter server
- https://127.0.1:8889/lab


Import packages


In [ ]:
from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create session


In [ ]:
spark = SparkSession.builder.appName("example").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create a dataframe


In [ ]:
data = [
    ("John", "Sales", 3000),
    ("Bryan", "Sales", 4200),
    ("Selena", "Sales", 4600),
    ("Alice", "Sales", 3500),
    ("Mark", "Sales", 3900),
    ("Sophia", "Sales", 4100),
    ("Daniel", "Sales", 3800),
    ("Emma", "Sales", 4400),
    ("Lucas", "Sales", 3600),
    ("Olivia", "Sales", 4700),
]
df = spark.createDataFrame(data, ["name", "department", "salary"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Visualize data


In [ ]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+------+
|  name|department|salary|
+------+----------+------+
|  John|     Sales|  3000|
| Bryan|     Sales|  4200|
|Selena|     Sales|  4600|
| Alice|     Sales|  3500|
|  Mark|     Sales|  3900|
|Sophia|     Sales|  4100|
|Daniel|     Sales|  3800|
|  Emma|     Sales|  4400|
| Lucas|     Sales|  3600|
|Olivia|     Sales|  4700|
+------+----------+------+

Filtering dataset (narrow)


In [ ]:
df_filtered = df.filter(df["salary"] > 4000)
df_filtered.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+------+
|  name|department|salary|
+------+----------+------+
| Bryan|     Sales|  4200|
|Selena|     Sales|  4600|
|Sophia|     Sales|  4100|
|  Emma|     Sales|  4400|
|Olivia|     Sales|  4700|
+------+----------+------+

Group By (Wide)


In [ ]:
df_grouped = df.groupby("department").count()
df_grouped.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|department|count|
+----------+-----+
|     Sales|   10|
+----------+-----+

## Pushdown Predicate

Reduce the total data read applying conditional filters to the data columns.

- Depends on file formats (Parquet and ORC)

1. Query submitted to spark
2. Planning phase using Catalyst optimizer
3. Filters identification (predicate)
4. Pushdown: try to bring the filter to reading

Using pure spark `WHERE` clause is enough, when using _AWS Glue_, `pushdown_predicate` argument is needed.


## Catalyst

Is a mechanism to optimize spark jobs utilizing functional programming principles to identify logical plan improvements (performance and execution time)

Catalyst has 19 rules (Spark 3+) to improve query execution.

Example:

- Pushdown predicate
- Rearrange filters
- Decimal operations conversion to long integer
- Regex rewriting in Java native code (startswith and contains)
- If-else simplification


In [11]:
spark = SparkSession.builder.appName("catalyst-join").getOrCreate()

employees_data = [
    (1, "John", "Sales", 3000),
    (2, "Bryan", "Sales", 4200),
    (3, "Selena", "Sales", 4600),
    (4, "Alice", "Sales", 3500),
    (5, "Mark", "Sales", 3900),
]

df_employees = spark.createDataFrame(
    employees_data, ["employee_id", "name", "department", "base_salary"]
)

sales_data = [
    (1, 120_000),
    (2, 180_000),
    (3, 220_000),
    (4, 95_000),
    (5, 160_000),
]

df_sales = spark.createDataFrame(sales_data, ["employee_id", "annual_sales"])

spark.sparkContext.setJobGroup(
    "JOIN", "Join employees with annual sales performance"
)

df_join_result = df_employees.join(df_sales, on="employee_id", how="inner")

df_join_result.show()

df_join_result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+----------+-----------+------------+
|employee_id|  name|department|base_salary|annual_sales|
+-----------+------+----------+-----------+------------+
|          1|  John|     Sales|       3000|      120000|
|          2| Bryan|     Sales|       4200|      180000|
|          3|Selena|     Sales|       4600|      220000|
|          4| Alice|     Sales|       3500|       95000|
|          5|  Mark|     Sales|       3900|      160000|
+-----------+------+----------+-----------+------------+

== Physical Plan ==
AdaptiveSparkPlan (11)
+- Project (10)
   +- SortMergeJoin Inner (9)
      :- Sort (4)
      :  +- Exchange (3)
      :     +- Filter (2)
      :        +- Scan ExistingRDD (1)
      +- Sort (8)
         +- Exchange (7)
            +- Filter (6)
               +- Scan ExistingRDD (5)


(1) Scan ExistingRDD
Output [4]: [employee_id#186L, name#187, department#188, base_salary#189L]
Arguments: [employee_id#186L, name#187, department#188, base_salary#189L], MapParti